# Visualizar una simulacion

Se usa la informacion que salva el modelo

In [1]:
import numpy as np
import math
import networkx as nx
import pandas as pd
import time
import random
import colorsys
from matplotlib.ticker import MaxNLocator
from collections import Counter
import matplotlib.colors as mcolors
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import os
import pickle
import matplotlib
from functools import partial

matplotlib.use('TkAgg')
plt.ion()

## Cargar informacion de la simulacion

In [2]:
# nombres de los archivos a cargar
carpeta_info = "./Info_simulacion_simple/"

In [3]:
# nombres de archivos
path_info_grafo = carpeta_info + "info_grafo.pkl"
path_info_paths = carpeta_info + "info_paths.pkl"
path_info_bloqueos = carpeta_info + "info_bloqueos.pkl"
path_info_csv = carpeta_info + 'results_data_collector.csv'
DPI = 400

In [4]:
# cargar info grafo
with open(path_info_grafo, 'rb') as f:
    dict_info_grafo = pickle.load(f)
# separar componentes
G = dict_info_grafo["G"]
pos = dict_info_grafo["pos"]
proba_origen = dict_info_grafo["proba_origen"]
proba_destino = dict_info_grafo["proba_destino"]

# ver
print(f"Grafo con {G.number_of_nodes()} nodos y {G.number_of_edges()} aristas")

Grafo con 288 nodos y 336 aristas


In [5]:
# cargar info paths
with open(path_info_paths, 'rb') as f:
    dict_info_paths = pickle.load(f)

# ver
print(f"Se tiene info de trayectorias de {len(dict_info_paths)} agentes")

Se tiene info de trayectorias de 144 agentes


In [6]:
# sobre escribir los colores
for info_path in dict_info_paths.values():
    info_path['color'] = 'forestgreen'

In [7]:
# cargar info bloqueos
with open(path_info_bloqueos, 'rb') as f:
    info_bloqueos = pickle.load(f)
    
# ver
print(f"Se tiene info de {len(info_bloqueos)} bloqueos")


# hacer una lista de diccionarios, donde cada entrada sea de un nodo bloqueado
# entonces, cada diccionario tiene:
#         nodo: nodo que se esta bloqueando
#         step_inicio: step inicial en donde se bloquea
#         step_fin: step final donde se bloquea
lista_nodos_bloqueos = []
# iterar el los bloqueos
for bloqueo in info_bloqueos:
    # por cada nodo bloqueado
    for nodo in bloqueo['nodos_bloqueados']:
        # crear el diccionario de este nodo
        dict_nodo = {
            "nodo": nodo,
            "step_inicio": bloqueo['inicio_bloqueo'],
            "step_fin": bloqueo['fin_bloqueo']
        }
        lista_nodos_bloqueos.append(dict_nodo)
        
# ver 
print(f"Se tiene info de {len(lista_nodos_bloqueos)} nodos bloqueados")

Se tiene info de 2 bloqueos
Se tiene info de 12 nodos bloqueados


In [8]:
# cargar el csv
df_dc = pd.read_csv(path_info_csv, index_col = 0) 
df_dc.head(5)

,Numero de agentes,Agentes esperando
0,50,0
1,51,5
2,52,5
3,53,4
4,54,6


## Info basica de la animacion

Sea $F$ = frames_per_step


El frame 0 inicia el step 0, los siguientes frames 
se usan para llevar a los nodos de la poblacion inicial a llegar al nodo donde van a estar en el step1.
Se tarda $s$ frames, por lo que en el frame en el frame $s$ todos los agentes estan en el nodo equivalente al step 1.


Mas generalmente, en el frame $F s$ los agentes estan en el nodo equivalente al step $s$.
Luego, el frame $F s + 1$ es el primero que comienza a llevar a los agentes al nodo del step $s+1$,
este proceso lleva $F$ frames, por lo que en el frame $F s + F = F(s+1)$
todos los agentes estan en el nodo del step $s+1$

Si un path comienza en el step $s$, entonces el primer frame en el que apareze es $F s$,
si termina en el $s'$ el ultimo step en el que aparece es en el $F s'$.
Si tiene $k$ nodos (aparece en $k$ steps) entonces $s' - s = k-1$,
por lo que $F s' - F s = F (k-1)$ y entonces aparece en $F (k-1) + 1$ frames

## Calcular la trayectoria continua que sigue un path

In [9]:
def compute_trayectoria_continua(path, stops_per_edge, G, pos):
    '''
    Dada una secuencia de nodos "path" en el grafo "G"
    Hacer un generador que de la trayectoria continua,
    se tienen "stops_per_edge" pasos por cada arista, 
    se usan las coordenadas del "pos"
    
    i -> j
    Pi (1-t) + Pj (t)
    
    Si se tiene un path con longuitud k 
    (k nodos, trabajo con path definido por secuencia de nodos)
    y se tienen f stops_per_edge
    la longuitud de los stops en la trayectoria continua
    es igual a: f(k-1) + 1
    
    pues el primer frame esta en el primer nodo,
    y de ahi se tarda k-1 frames a llegar a un nodo, y faltan k-1
    '''
    
    # tomar numeros usados para interpolar linealmente los nodos
    parametro_t_simple = np.linspace(0, 1, stops_per_edge+1)[1:]
    # poner en dos columnas, para usar en dos coordenadas
    parametro_t = np.array([parametro_t_simple, parametro_t_simple]).T

    # ir guardando todas las coordenads en el camino continuo
    # el primer stop obviamente es la coordenada del primer nodo
    stops = np.array(pos[path[0]]).reshape(1, -1)

    # iterar en cada arista del path (i -> j)
    for nodo_i, nodo_j in zip(path, path[1:]):

        # sacar coordenadas de los nodos i,j
        # i.e puntos en el plano
        P_i = np.array(pos[nodo_i]).reshape(1, -1)
        P_j = np.array(pos[nodo_j]).reshape(1, -1)

        # calcular los stops que van de i a j
        # obvio es la combinacion convexa de los puntos
        new_stops = P_i * (1 - parametro_t) + P_j * (parametro_t)

        # agregar
        stops = np.vstack((stops, new_stops))
    
    # devolver
    return stops

## Actualizar la imagen en cada frame

In [10]:
# la funcion que se llama iterativamente
def update_agents_args(frame, frames_per_step, dict_figuras_agentes, lista_nodos_bloqueos,
                       dict_info_paths):
    '''
    Funcion que se llama durante la animacion
    Se llama para cada valor de frame (empezando en 0)
    Se consideran bloqueos
    '''
    
    # usando el frame actual, y los frames por segundo
    # se puede saber en que step actual se esta
    # notar que puede ser decimal, pero da igual
    step_actual = frame/frames_per_step
    
    # poner en una lista los ciruclos a actualizar
    updated_patches = []

    # ------------------------------------------------------------------------------
    
    # Actualizar caminos
    
    # iterar en todos los caminos
    for key_path, info_path in dict_info_paths.items():
        
         # sacar el circulo de este agente
        circle_agent = dict_figuras_agentes[key_path]
        
        # solo meterse con caminos que existane en este step
        if info_path['step_creation'] <= step_actual <= info_path['last_step']:
            
            # si no es visible pues hacer visible
            if not circle_agent.get_visible():
                circle_agent.set_visible(True)
                
            # ver su posicion usando trayectoria continua (usando el frame actual)
            nueva_stop = info_path["trayectoria_completa"][frame]
            
            # actualizar
            circle_agent.center = nueva_stop
            updated_patches.append(circle_agent)
             
        # end if, el camino existe en este step
        
        # de otro modo, si ya dejo de existir, pero siguie visible
        elif info_path['last_step'] < step_actual and circle_agent.get_visible():
            # hacer invisible
            circle_agent.set_visible(False)
            updated_patches.append(circle_agent)
                
    # ------------------------------------------------------------------------------
    
    # Actualizar bloqueos
    
    # iterar en cada nodo bloqueado
    for nodo_boqueado in lista_nodos_bloqueos:
        
        # tomar la figura
        fig_nodo_bloqueado = nodo_boqueado["figura"]
        
        # si le toca en este step  y no es visible
        if nodo_boqueado['step_inicio'] <= step_actual <= nodo_boqueado['step_fin'] and not fig_nodo_bloqueado.get_visible():
            # hacer visible
            fig_nodo_bloqueado.set_visible(True)
            updated_patches.append(fig_nodo_bloqueado)
            
        # si no le toca y sigue siendo visible
        elif nodo_boqueado['step_fin'] < step_actual and fig_nodo_bloqueado.get_visible():
            # hacer invisible
            fig_nodo_bloqueado.set_visible(False)
            updated_patches.append(fig_nodo_bloqueado)
                
                     
    return updated_patches

## Funcion principal de la animacion

In [11]:
def animar_simulacion(frames_per_step = 30,  seconds_per_step = 1,
                      node_size = 50, arrowsize = 10,
                      node_color = '#f0f0f0', edge_color = 'gray',
                      radius_agentes = 0.05, alpha_agentes = 1,
                      color_bloqueo = "red", radius_bloqueo = 0.05,
                      name_salvar = "prueba"):
    
    '''
    Anima la simulacion basada en agentes, muestra las trayectorias de los agentes
    
    frames_per_step - cuantos frames de la animacion suceden en cada step del modelo
    seconds_per_step - cuantos segundos se tarda un agente de ir de un nodo a otro
    
    node_size - tamaño de los nodos del grafo
    arrowsize - tamaño de las flechas del grafo
    node_color - color de los nodos del grafo
    edge_color - color de las aristas del grafo
    
    radius_agentes - radio de los agentes
    alpha_agentes - para la transparencia de los agentes
    '''
    
    
    # calcular el ultimo step para cada path, y ponerlo como atributo
    for info_path in dict_info_paths.values():

        # el ultimo step se calcula con el primero
        # y la longuitud del camino (-1 obvio)
        last_step = info_path['step_creation'] + len(info_path['path']) - 1

        # agregar
        info_path['last_step'] = last_step

        
    # ver el mayor de todos, este es el ultimo step considerado en la animacion
    last_step = max([info_path['last_step'] for info_path in dict_info_paths.values()])
    
    # calcular cuantos frames va a tener la animacion
    FRAMES = last_step * frames_per_step    
    
    # -----------------------------------------------------------------------------------------
    
    # guardar el generador de cada path
    # guardarlo como un diccionario,
    # donde el frame correspondiente es la llave
    # y el valor es la posicion en ese frame
    
    # iterar en los caminos
    for key_path, info_path in dict_info_paths.items():
        
        # sacar la trayectoria continua
        trayectoria_continua_path = compute_trayectoria_continua(path = info_path["path"],
                                                                 stops_per_edge = frames_per_step,
                                                                 G = G, pos=pos)
        
        # calcular cual es el frame inicial de este agetne
        frame_inicial_agente = info_path['step_creation'] * frames_per_step
        
        # hacer un dict donde los frames sean las llaves y la posicion el valor
        dict_trayectoria_continua_path = {frame_inicial_agente + idx: posicion
                                          for idx, posicion in enumerate(trayectoria_continua_path)}

        # ponerlo como atributo
        dict_info_paths[key_path]["trayectoria_completa"] = dict_trayectoria_continua_path
        
        
    # -----------------------------------------------------------------------------------------
    
    fig, ax = plt.subplots(figsize=(6, 6), dpi = DPI)
        
        
    # hacer el grafo
    nx.draw(G, pos, ax=ax, node_size=node_size,  arrowsize = arrowsize,
            node_color = node_color, edge_color = edge_color)
    plt.tight_layout()

    
    
    # por cada agente, hacer una figura y ponerlo no visible
    dict_figuras_agentes = dict()

    # iterar en cada path
    for key_path, info_path in dict_info_paths.items():
        # hacer la figura de este agente
        circulo_agente = plt.Circle(xy = (0, 0), color = info_path['color'],
                                    radius = radius_agentes, alpha = alpha_agentes,
                                    zorder = 2)
        # añadirla al ax como no visible
        ax.add_patch(circulo_agente)
        circulo_agente.set_visible(False)
        # guardarla usando la key del path
        dict_figuras_agentes[key_path] = circulo_agente
        
        
                    
    # por cada nodo bloqueado, hacer una figura en ese nodo y hacerlo no visible
    for nodo_bloqueado in lista_nodos_bloqueos:
        # hacer la figura de este nodo bloqueado
        circulo_bloqueo = plt.Circle(xy = pos[nodo_bloqueado["nodo"]],
                                     color = color_bloqueo,
                                     radius = radius_bloqueo, alpha = 1,
                                     zorder = 2)
        # añadirla al ax como no visible
        ax.add_patch(circulo_bloqueo)
        circulo_bloqueo.set_visible(False)
        # guardarla en el dict del nodo
        nodo_bloqueado["figura"] = circulo_bloqueo
        
        
    # ----------------------------------------------------------------------------------------

    # Definir la funcion que actualiza cada frame
    # es la que ya se tiene, pero con los parametros adecuandos
    
    update_agents = partial(update_agents_args,
                            frames_per_step = frames_per_step,
                            dict_figuras_agentes = dict_figuras_agentes,
                            lista_nodos_bloqueos = lista_nodos_bloqueos,
                            dict_info_paths = dict_info_paths)
    
        
    # ----------------------------------------------------------------------------------------
        
    # calcular cual es el interval (Milisegundos de espera entre cada frame)
    interval = max(1, int(1000 * seconds_per_step / frames_per_step))
        
    # Configurar la animación
    # interval controla el tiempo en milisegundos entre llamadas a la funcion
    ani = animation.FuncAnimation(fig, update_agents, repeat=False, blit=True,
                                  frames= FRAMES+2, interval = interval)
    
    # salvar
    # honestamente no entiendo como, pero sin esta linea no funciona
    plt.rcParams['animation.ffmpeg_path'] = 'C:\\Users\\diego\\anaconda3\\envs\\visua\\Library\\bin\\ffmpeg'
    fps = int(frames_per_step/seconds_per_step)
    ani.save(name_salvar + '.mp4', writer='ffmpeg', fps=fps)
    
    plt.close()

# Animacion completa

In [12]:
# la funcion que se llama iterativamente
def update_agents_completo_args(frame, frames_per_step, dict_figuras_agentes, lista_nodos_bloqueos,
                                dict_info_paths, df_dc, line_num_agentes, line_agentes_esperando):
    '''
    Funcion que se llama durante la animacion
    Se llama para cada valor de frame (empezando en 0)
    
    La animacion tiene el mapa y ax con graficas
    Se consideran bloqueos
    '''
    
    # usando el frame actual, y los frames por segundo
    # se puede saber en que step actual se esta
    # notar que puede ser decimal, pero da igual
    step_actual = frame/frames_per_step
    
    # poner en una lista los ciruclos a actualizar
    updated_patches = []

    
    # ------------------------------------------------------------------------------
    
    # Actualizar caminos
    
    # iterar en todos los caminos
    for key_path, info_path in dict_info_paths.items():
        
         # sacar el circulo de este agente
        circle_agent = dict_figuras_agentes[key_path]
        
        # solo meterse con caminos que existane en este step
        if info_path['step_creation'] <= step_actual <= info_path['last_step']:
            
            # si no es visible pues hacer visible
            if not circle_agent.get_visible():
                circle_agent.set_visible(True)
                
            # ver su posicion usando trayectoria continua (usando el frame actual)
            nueva_stop = info_path["trayectoria_completa"][frame]
            
            # actualizar
            circle_agent.center = nueva_stop
            updated_patches.append(circle_agent)
             
        # end if, el camino existe en este step
        
        # de otro modo, si ya dejo de existir, pero siguie visible
        elif info_path['last_step'] < step_actual and circle_agent.get_visible():
            # hacer invisible
            circle_agent.set_visible(False)
            updated_patches.append(circle_agent)
            
            
    # ------------------------------------------------------------------------------
    # Actualizar bloqueos
    
    # iterar en cada nodo bloqueado
    for nodo_boqueado in lista_nodos_bloqueos:
        
        # tomar la figura
        fig_nodo_bloqueado = nodo_boqueado["figura"]
        
        # si le toca en este step  y no es visible
        if nodo_boqueado['step_inicio'] <= step_actual <= nodo_boqueado['step_fin'] and not fig_nodo_bloqueado.get_visible():
            # hacer visible
            fig_nodo_bloqueado.set_visible(True)
            updated_patches.append(fig_nodo_bloqueado)
            
        # si no le toca y sigue siendo visible
        elif nodo_boqueado['step_fin'] < step_actual and fig_nodo_bloqueado.get_visible():
            # hacer invisible
            fig_nodo_bloqueado.set_visible(False)
            updated_patches.append(fig_nodo_bloqueado)
                
    
    # ------------------------------------------------------------------------------
    # Actualizar las graficas
    
    # ahora tener el stepa actual como int
    step_actual = int(frame/frames_per_step)
    
    # actualzizar datos de numero de agentes
    line_num_agentes.set_data(df_dc.index[:(step_actual+1)], df_dc["Numero de agentes"][:(step_actual+1)])
    # ponerlo en las cosas que se actualizan
    updated_patches.append(line_num_agentes)
    
    # actualizar datos de agentes esperando
    line_agentes_esperando.set_data(df_dc.index[:(step_actual+1)], df_dc["Agentes esperando"][:(step_actual+1)])
    # ponerlo en las cosas que se actualizan
    updated_patches.append(line_agentes_esperando)
        
                     
    return updated_patches

In [13]:
def animar_simulacion_completo(frames_per_step = 30,  seconds_per_step = 1,
                              node_size = 50, arrowsize = 10,
                              node_color = '#f0f0f0', edge_color = 'gray',
                              radius_agentes = 0.05, alpha_agentes = 1,
                              color_bloqueo = "red", radius_bloqueo = 0.05,
                              name_salvar = "prueba"):
    
    '''
    Anima la simulacion basada en agentes, muestra las trayectorias de los agentes
    
    frames_per_step - cuantos frames de la animacion suceden en cada step del modelo
    seconds_per_step - cuantos segundos se tarda un agente de ir de un nodo a otro
    
    node_size - tamaño de los nodos del grafo
    arrowsize - tamaño de las flechas del grafo
    node_color - color de los nodos del grafo
    edge_color - color de las aristas del grafo
    
    radius_agentes - radio de los agentes
    alpha_agentes - para la transparencia de los agentes
    '''
    
    # calcular el ultimo step para cada path, y ponerlo como atributo
    for info_path in dict_info_paths.values():

        # el ultimo step se calcula con el primero
        # y la longuitud del camino (-1 obvio)
        last_step = info_path['step_creation'] + len(info_path['path']) - 1

        # agregar
        info_path['last_step'] = last_step

        
    # ver el mayor de todos, este es el ultimo step considerado en la animacion
    last_step = max([info_path['last_step'] for info_path in dict_info_paths.values()])
    
    # calcular cuantos frames va a tener la animacion
    FRAMES = last_step * frames_per_step
    
    # -----------------------------------------------------------------------------------------
    
    # guardar cada path
    # guardarlo como un diccionario,
    # donde el frame correspondiente es la llave
    # y el valor es la posicion en ese frame
    
    # iterar en los caminos
    for key_path, info_path in dict_info_paths.items():
        
        # sacar la trayectoria continua
        trayectoria_continua_path = compute_trayectoria_continua(path = info_path["path"],
                                                                 stops_per_edge = frames_per_step,
                                                                 G = G, pos=pos)
        
        # calcular cual es el frame inicial de este agetne
        frame_inicial_agente = info_path['step_creation'] * frames_per_step
        
        # hacer un dict donde los frames sean las llaves y la posicion el valor
        dict_trayectoria_continua_path = {frame_inicial_agente + idx: posicion
                                          for idx, posicion in enumerate(trayectoria_continua_path)}

        # ponerlo como atributo
        dict_info_paths[key_path]["trayectoria_completa"] = dict_trayectoria_continua_path
        
        
    # -----------------------------------------------------------------------------------------
    
    # hacer la figura, es un mosaico

    fig = plt.figure(figsize=(10, 5), layout="constrained", dpi = DPI)
    axd = fig.subplot_mosaic([["Map", "Up"],
                              ["Map", "Down"]])


    # ---------------
    # Mapa
    nx.draw(G, pos, ax=axd["Map"], node_size=node_size,  arrowsize = arrowsize,
            node_color = node_color, edge_color = edge_color)
    plt.tight_layout()
    
    
    # por cada agente, hacer una figura y ponerlo no visible
    dict_figuras_agentes = dict()
    # iterar en cada path
    for key_path, info_path in dict_info_paths.items():
        # hacer la figura de este agente
        circulo_agente = plt.Circle(xy = (0, 0), color = info_path['color'],
                                    radius = radius_agentes, alpha = alpha_agentes,
                                    zorder = 2)
        # añadirla al ax como no visible
        axd["Map"].add_patch(circulo_agente)
        circulo_agente.set_visible(False)
        # guardarla usando la key del path
        dict_figuras_agentes[key_path] = circulo_agente
        
    # por cada nodo bloqueado, hacer una figura en ese nodo y hacerlo no visible
    for nodo_bloqueado in lista_nodos_bloqueos:
        # hacer la figura de este nodo bloqueado
        circulo_bloqueo = plt.Circle(xy = pos[nodo_bloqueado["nodo"]],
                                     color = color_bloqueo,
                                     radius = radius_bloqueo, alpha = 1,
                                     zorder = 2)
        # añadirla al ax como no visible
        axd["Map"].add_patch(circulo_bloqueo)
        circulo_bloqueo.set_visible(False)
        # guardarla en el dict del nodo
        nodo_bloqueado["figura"] = circulo_bloqueo
        
        
        
    # por cada nodo bloqueado, hacer una figura en el nodo, y ponerlo no visible
    
        
    # ---------------
    # Grafica de numero de agentes
    
    # inicia vacia
    line_num_agentes, = axd["Up"].plot([], [])
    
    # diseño
    axd["Up"].set_xlim([df_dc.index[0], df_dc.index[-1]])  # limite en x
    axd["Up"].set_ylim([df_dc["Numero de agentes"].min(), df_dc["Numero de agentes"].max()]) # limite en y
    axd["Up"].spines['top'].set_visible(False)  # no ver linea
    axd["Up"].spines['right'].set_visible(False) # no ver linea
    axd["Up"].set_xlabel("Step")
    axd["Up"].set_ylabel("Coches")
    axd["Up"].set_title("Numero de agentes en el mapa")
    axd["Up"].xaxis.set_major_locator(MaxNLocator(integer=True)) # solo ticks enteros en x
    axd["Up"].yaxis.set_major_locator(MaxNLocator(integer=True)) # solo ticks enteros en y
    
    # ---------------
    # Grafica de agentes esperando
    
    # inicia vacia
    line_agentes_esperando, = axd["Down"].plot([], [])
    
    # diseño
    axd["Down"].set_xlim([df_dc.index[0], df_dc.index[-1]])  # limite en x
    axd["Down"].set_ylim([df_dc["Agentes esperando"].min(), df_dc["Agentes esperando"].max()]) # limite en y
    axd["Down"].spines['top'].set_visible(False)  # no ver linea
    axd["Down"].spines['right'].set_visible(False) # no ver linea
    axd["Down"].set_xlabel("Step")
    axd["Down"].set_ylabel("Coches")
    axd["Down"].set_title("Numero de agentes esperando")
    axd["Down"].xaxis.set_major_locator(MaxNLocator(integer=True)) # solo ticks enteros en x
    axd["Down"].yaxis.set_major_locator(MaxNLocator(integer=True)) # solo ticks enteros en y
        
            
    # ----------------------------------------------------------------------------------------

    # Definir la funcion que actualiza cada frame
    # es la que ya se tiene, pero con los parametros adecuandos
    
    update_agents = partial(update_agents_completo_args,
                            frames_per_step = frames_per_step,
                            dict_figuras_agentes = dict_figuras_agentes,
                            lista_nodos_bloqueos = lista_nodos_bloqueos,
                            dict_info_paths = dict_info_paths,
                            df_dc = df_dc,
                            line_num_agentes = line_num_agentes,
                            line_agentes_esperando = line_agentes_esperando)
    
        
    # ----------------------------------------------------------------------------------------
        
    # calcular cual es el interval (Milisegundos de espera entre cada frame)
    interval = max(1, int(1000 * seconds_per_step / frames_per_step))

        
    # Configurar la animación
    # interval controla el tiempo en milisegundos entre llamadas a la funcion
    ani = animation.FuncAnimation(fig, update_agents, repeat=False, blit=True,
                                  frames= FRAMES+2, interval = interval)
    
    
    # salvar
    # honestamente no entiendo como, pero sin esta linea no funciona
    plt.rcParams['animation.ffmpeg_path'] = 'C:\\Users\\diego\\anaconda3\\envs\\visua\\Library\\bin\\ffmpeg'
    fps = int(frames_per_step/seconds_per_step)
    ani.save(name_salvar + '.mp4', writer='ffmpeg', fps=fps)
    
    plt.close()

## Funcion que junta ambas opciones

In [14]:
def hacer_animacion(animacion_completa = True,
                    frames_per_step = 30,  seconds_per_step = 1,
                    node_size = 50, arrowsize = 10,
                    node_color = '#f0f0f0', edge_color = 'gray',
                    radius_agentes = 0.05, alpha_agentes = 1,
                    color_bloqueo = "red", radius_bloqueo = 0.05,
                    name_salvar = "prueba"):
    '''
    Hace la animacion, completa o no
    '''
    
    if animacion_completa:
        # llamar a la funcion
        animar_simulacion_completo(frames_per_step = frames_per_step, seconds_per_step = seconds_per_step,
                                   node_size = node_size, arrowsize = arrowsize,
                                   node_color = node_color, edge_color = edge_color,
                                   radius_agentes = radius_agentes, alpha_agentes = alpha_agentes,
                                   color_bloqueo = color_bloqueo, radius_bloqueo = radius_bloqueo,
                                   name_salvar = name_salvar)
        
    else:
        # llamar a la funcion
        animar_simulacion(frames_per_step = frames_per_step, seconds_per_step = seconds_per_step,
                          node_size = node_size, arrowsize = arrowsize,
                          node_color = node_color, edge_color = edge_color,
                          radius_agentes = radius_agentes, alpha_agentes = alpha_agentes,
                          color_bloqueo = color_bloqueo, radius_bloqueo = radius_bloqueo,
                          name_salvar = name_salvar)
    

# Ejecutar codigo

In [15]:
# ver la animacion grafo simple
hacer_animacion(animacion_completa = False,
                frames_per_step = 5, seconds_per_step = 0.5,
                node_size = 100, arrowsize = 10,
                node_color = '#c0c0c0', edge_color = "#3f3f3f",
                radius_agentes = 0.3, alpha_agentes = 1,
                color_bloqueo = "red", radius_bloqueo = 0.3,
                name_salvar = "./Videos_ejemplo/animacion_grafo_simple")

In [16]:
# # ver la animacion
# hacer_animacion(animacion_completa = False,
#                 frames_per_step = 2, seconds_per_step = 0.2,
#                 node_size = 0.1, arrowsize = 0.1,
#                 node_color = '#f0f0f0', edge_color = 'gray',
#                 radius_agentes = 5, alpha_agentes = 1,
#                 color_bloqueo = "red", radius_bloqueo = 0.2,
#                 name_salvar = "./Videos_ejemplo/animacion_grafo_simple")